In [1]:

from sandbox.specs import (
    MAP_REPO_TO_REQS_PATHS,
    MAP_REPO_VERSION_TO_SPECS_PY,
)
from sandbox import bash_session, docker_build

In [2]:
from datasets import load_dataset

dataset = load_dataset("lahirum/SWE_Experimental", split="train")

/home/lahiru-menik/miniconda3/envs/agentless/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAP_REPO_VERSION_TO_SPECS_PY[dataset[0]['repo']][dataset[0]['version']]

{'python': '3.9',
 'packages': 'requirements.txt',
 'install': 'python -m pip install -e .',
 'test_cmd': './tests/runtests.py --verbosity 2'}

In [23]:
import docker

def image_exists(name):
    # client = docker.from_env()
    client = docker.DockerClient(base_url='unix:///home/lahiru-menik/.docker/desktop/docker.sock')
    print(client.images.list(), "lll")
    try:
        client.images.get(name)
        return True
    except docker.errors.ImageNotFound:
        return False
    except docker.errors.APIError as e:
        print(f"Docker API error: {e}")
        return False

In [24]:
print(image_exists("django_4.1"))

[<Image: 'django_4.1:latest'>, <Image: 'django-env:4.2'>, <Image: 'swe/310:latest'>, <Image: 'httpd:2.4'>, <Image: 'hello-world:latest'>] lll
True


In [22]:

client = docker.DockerClient(base_url='unix:///home/lahiru-menik/.docker/desktop/docker.sock')
images = client.images.list()
print(images)

[<Image: 'django_4.1:latest'>, <Image: 'django-env:4.2'>, <Image: 'swe/310:latest'>, <Image: 'httpd:2.4'>, <Image: 'hello-world:latest'>]


In [18]:
import docker

client = docker.from_env()
images = client.images.list()

print("=== Docker SDK sees the following images ===")
for image in images:
    print(image.id)
    print("Tags:", image.tags)
    print("-----")

=== Docker SDK sees the following images ===
sha256:74cc54e27dc41bb10dc4b2226072d469509f2f22f1a3ce74f4a59661a1d44602
Tags: ['hello-world:latest']
-----


In [19]:
import docker
client = docker.from_env()
print("Docker SDK connected to:", client.api.base_url)


Docker SDK connected to: http+docker://localhost


In [25]:
from commit import update
import os
import tempfile
from pathlib import Path
import pexpect

for i in range(1):
    commit_id = dataset[i]['base_commit']
    fail_to_passs = dataset[i]['FAIL_TO_PASS']
    version = dataset[i]['version']
    pass_to_pass = dataset[i]['PASS_TO_PASS']
    name = dataset[i]['instance_id'].split("__")[0]
    # print(name)
    update(name, commit_id)
    bash = bash_session.BashSession()
    bash.run_command(f"cp -r {name} testbed")
    host_dir = os.path.abspath(".") + "/testbed/" + name
    container_dir = "/app" 
    exist_check = image_exists(name + "_" + version)
    print(exist_check)
    specs = MAP_REPO_VERSION_TO_SPECS_PY[dataset[i]['repo']][dataset[i]['version']]
    # exist_check = bash.run_command(f"docker images | grep -q '^{name+'_'+version}' && echo yes || echo no")
    if not exist_check:
        print(f"Building image {name}_{version}...")
        if "pip_packages" in specs:
            pip_packages = " ".join(specs["pip_packages"])
        else:
            pip_packages = ""
        if "pre_install" in specs:
            pre_install_commands = "\n".join(specs["pre_install"])
        else:
            pre_install_commands = ""
        dockerfile_str = docker_build.generate_dockerfile(
            python_version= specs["python"],
            pre_install= pre_install_commands,
            pip_packages=pip_packages,
            
        )
        
        tmpdir = tempfile.mkdtemp()
        Path(f"{tmpdir}/Dockerfile").write_text(dockerfile_str)
        print(f"Generated Dockerfile for {name}_{version}")
        bash.run_command(f"docker build -t {name}_{version} {tmpdir}")
        bash.logfile = open(f"{name}_{version}_dockerbuild.log", "w")
      
    bash.run_command(f"docker run --mount type=bind,src={host_dir},dst={container_dir},ro -it {name}_{version} bash")
    print(bash.run_command("ls -la"))
        
        
    
    
    

django
DiGraph with 27538 nodes and 40138 edges
Checked out to 4fd3044ca0135da903a70dfb66992293f529ecf1
DiGraph with 27538 nodes and 40138 edges
[<Image: 'django_4.1:latest'>, <Image: 'django-env:4.2'>, <Image: 'swe/310:latest'>, <Image: 'httpd:2.4'>, <Image: 'hello-world:latest'>] lll
True
a
total 184
drwxrwxr-x  12 sandbox sandbox  4096 May 27 15:24 .
drwxr-xr-x   1 root    root     4096 May 27 18:58 ..
-rw-rw-r--   1 sandbox sandbox   807 May 27 18:58 .editorconfig
-rw-rw-r--   1 sandbox sandbox   120 May 27 18:58 .eslintignore
-rw-rw-r--   1 sandbox sandbox  1407 May 27 18:58 .eslintrc
drwxrwxr-x   8 sandbox sandbox  4096 May 27 15:24 .git
-rw-rw-r--   1 sandbox sandbox   297 May 27 18:58 .gitattributes
drwxrwxr-x   3 sandbox sandbox  4096 May 27 15:24 .github
-rw-rw-r--   1 sandbox sandbox   353 May 27 18:58 .gitignore
-rw-rw-r--   1 sandbox sandbox   281 May 27 18:58 .pre-commit-config.yaml
-rw-rw-r--   1 sandbox sandbox   385 May 27 18:58 .readthedocs.yml
drwxrwxr-x   2 sandbox 